# Network with 2 GRU Layers:

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import array
from keras.models import load_model

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
from keras.callbacks import ModelCheckpoint

# Data Preparation:

In [3]:
#read all the data:
items = pd.read_csv("item_features.csv")
purchase =  pd.read_csv("train_purchases.csv")
sessions =  pd.read_csv("train_sessions.csv")

In [4]:
import time
purchase['date'] = pd.to_datetime(purchase['date'])
sessions['date'] = pd.to_datetime(sessions['date'])
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [5]:
max_day = sessions['date'].max() #max date 31/05/2021 
testing_date = max_day -datetime.timedelta(31)
cond_test = sessions['date'] >= testing_date
cond_train = sessions['date'] < testing_date
cond_test_purchase = purchase.date >= testing_date
cond_train_purchase = purchase.date < testing_date
sessions_train = sessions[cond_train]
sessions_test = sessions[cond_test]
purchase_train = purchase[cond_train_purchase]
purchase_test = purchase[cond_test_purchase]

In [6]:
def sessions_list(sessions,purchase):
    antique_session = sessions.iat[0,0]
    list_sessions = []
    actual_session = []
    purchase_session = 0
    for i in tqdm(range(len(sessions))):
        new_session = sessions.iat[i,0]
        item = sessions.iat[i,1]
        if(new_session != antique_session):
            actual_session.append(purchase.iat[purchase_session,1])
            purchase_session += 1
            list_sessions.append(actual_session)
            actual_session = []
            antique_session = new_session
        actual_session.append(item)
    list_sessions.append(actual_session)
    return list_sessions

In [7]:
session_item = sessions_list(sessions,purchase)

100%|██████████| 4743820/4743820 [04:26<00:00, 17767.32it/s]


In [8]:
def training_sequences(session_item2):
    train_sequence = np.zeros((len(session_item2),6))
    trobat = False
    for i,x in tqdm(enumerate(session_item2)):
        length = len(x)
        llista = x.copy()
        if(length-1 < 5):
            for j in range(5-length+1):
                llista.insert(0,0)
        if(length-1 > 5):
            for j in range(length-6):
                llista.pop(0)
        llista.pop(-1)                                    
        train_sequence[i][0] = llista[0]
        train_sequence[i][1] = llista[1]
        train_sequence[i][2] = llista[2]
        train_sequence[i][3] = llista[3]
        train_sequence[i][4] = llista[4]
        train_sequence[i][5] = x[len(x)-1]
    return train_sequence

sequence_training = training_sequences(session_item)

1000000it [00:04, 235695.46it/s]


In [9]:
X = np.array(sequence_training)[:,:5] #sequence to predict from
y = np.array(sequence_training)[:,5:] #next item in the sequence
X = X.astype(int)
y = y.astype(int)
training = np.array(sequence_training)

# Network:

In [10]:
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [11]:
number_items = len(items.item_id.unique())
number_items += 1
number_items

23692

In [12]:
item_index = {}
index_item = {}
item_index[0] = 0
index_item[0] = 0
for i,x in enumerate(items.item_id.unique()):
    index_item[i+1] = x
    item_index[x] = i+1

In [13]:
def training_dataset(X2,y2):
    #first X.
    X_training = np.zeros((X2.shape[0],X2.shape[1]))
    y_training = np.zeros(y2.shape[0])
    
    for i in range(X2.shape[0]):
        for j in range(X2.shape[1]):
            X_training[i,j] = item_index[X2[i,j]]
    
    for i in range(y2.shape[0]):
        y_training[i] = item_index[y2[i][0]]
            
    return X_training, y_training
X_training, y_training = training_dataset(X,y)  
X_training = X_training.astype(int)
y_training = y_training.astype(int)

In [16]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 128, input_length=input_len))
    model.add(GRU(256,return_sequences=True))
    model.add(Dropout(0.2))
    # Add Hidden Layer 1 - LSTM Layer
    model.add(GRU(128))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    
    return model

In [17]:
#let's define a sparse categorical crossentropy to start!
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

In [18]:
#let's define a sparse categorical crossentropy to start!
def loss2(labels, logits):
  print(labels)
  labels = int(labels)
  index = item_index[labels]
  return 1
  logits2 = np.array(logits)
  return 1
  res = list(sorted(enumerate(test_list)))[-100:]
  for i in range(100):
    if(index == res[i][0]): return 1
  return 1

**Metrics**

In [19]:
#let's define a sparse categorical crossentropy to start!
from tensorflow.keras.utils import to_categorical
def top_5(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=5
)
def top_50(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=50
)
def top_100(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=100
)

**Model Compilation**

In [20]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.38.145.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.145.154:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [21]:
with strategy.scope():
  model = create_model(6, number_items)
  adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(
      optimizer=adam_optimizer,
      loss=loss,
      metrics= ["acc",top_5,top_100],
      steps_per_execution=32
  )
model.fit(X_training,y_training,epochs=100, batch_size=128, validation_split = 0.01, verbose=1,
          callbacks  = [EarlyStopping(monitor='val_acc', patience=3),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True)]
            )
model.save_weights('/tmp/best.h5', overwrite=True)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7735/7735 [==============================] - 68s 9ms/step - loss: 8.0849 - acc: 0.0223 - top_5: 0.0612 - top_100: 0.2498 - val_loss: 7.2124 - val_acc: 0.0461 - val_top_5: 0.1219 - val_top_100: 0.3819
Epoch 2/100
7735/7735 [==============================] - 49s 6ms/step - loss: 7.0588 - acc: 0.0513 - top_5: 0.1302 - top_100: 0.4097 - val_loss: 6.8086 - val_acc: 0.0679 - val_top_5: 0.1639 - val_top_100: 0.4544
Epoch 3/100
7735/7735 [==============================] - 49s 6ms/step - loss: 6.7243 - acc: 0.0654 - top_5: 0.1601 - top_100: 0.4681 - val_loss: 6.6471 - val_acc: 0.0810 - val_top_5: 0.1823 - val_top_100: 0.4868
Epoch 4/100
7735/7735 [==============================] - 49s 6ms/step - loss: 6.5208 - acc: 0.0756 - top_5: 0.1796 - top_100: 0.5029 - val_loss: 6.5723 - val_acc: 0.0829 - val_top_5: 0.1899 - val_top_100: 0.5027
Epoch 5/100
7735/7735 [==============================] - 49s 6ms/step - loss: 6.3707 - acc: 0.0828 - top_5: 0.1943 - top_100: 0.5265 - val_loss: 6.5201 - val_acc: 0

# Obtaining Predictions:

In [ ]:
model = tf.keras.models.load_model('best_model.h5', compile=False)

#let's define a sparse categorical crossentropy to start!
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

# summarize model.
model.summary()
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
      optimizer=adam_optimizer,
      loss=loss,
      metrics= ["acc",top_5,top_100],
      steps_per_execution=32
  )

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 128)            3032576   
                                                                 
 lstm_2 (LSTM)               (None, 5, 256)            394240    
                                                                 
 dropout_2 (Dropout)         (None, 5, 256)            0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 23692)             3056268   
                                                                 
Total params: 6,680,204
Trainable params: 6,680,204
No

In [ ]:
leaderboard = pd.read_csv("test_leaderboard_sessions.csv")

In [ ]:
def sessions_list_testing(sessions):
    antique_session = sessions.iat[0,0]
    list_sessions = []
    actual_session = []
    for i in tqdm(range(len(sessions))):
        new_session = sessions.iat[i,0]
        item = sessions.iat[i,1]
        if(new_session != antique_session):
            list_sessions.append(actual_session)
            actual_session = []
            antique_session = new_session
        actual_session.append(item)
    list_sessions.append(actual_session)
    return list_sessions

In [ ]:
session_item_testing = sessions_list_testing(leaderboard)

100%|██████████| 229354/229354 [00:18<00:00, 12603.99it/s]


In [ ]:
def testing_sequences(session_item2):
    train_sequence = np.zeros((len(session_item2),5))
    trobat = False
    for i,x in tqdm(enumerate(session_item2)):
        length = len(x)
        llista = x.copy()
        if(length < 5):
            for j in range(5-length):
                llista.insert(0,0)
        if(length > 5):
            for j in range(length-5):
                llista.pop(0)
        train_sequence[i][0] = llista[0]
        train_sequence[i][1] = llista[1]
        train_sequence[i][2] = llista[2]
        train_sequence[i][3] = llista[3]
        train_sequence[i][4] = llista[4]
    return train_sequence

sequence_testing = testing_sequences(session_item_testing)

50000it [00:00, 101768.27it/s]


In [ ]:
X_testing = np.array(sequence_testing)
X_testing = X_testing.astype(int)

In [ ]:
def testing_dataset(X2):
    #first X.
    X_test = np.zeros((X2.shape[0],X2.shape[1]))    
    for i in range(X2.shape[0]):
        for j in range(X2.shape[1]):
            X_test[i,j] = item_index[X2[i,j]]

    return X_test
X_leaderboard = testing_dataset(X_testing)  
X_leaderboard = X_leaderboard.astype(int)

**Creating The prediction Fully with LSTM:**

In [ ]:
def generate_solution(X_test,leaderboard2, split = 100):
    solution = [[]]
    sessions_unique = leaderboard.session_id.unique()
    for i in tqdm(range(int(50000/split))):
      prediction = model.predict(X_test[split*i:split*(i+1)])
      for j in range(split):
        prediction_actual = prediction[j]
        indexes = np.argpartition(prediction_actual, -100)[-100:]
        best_100 = indexes[np.argsort(-prediction_actual[indexes])]
        for k in range(100):
          solution.append([int(sessions_unique[j+i*split]),index_item[int(best_100[k])],k+1])

    return solution


In [ ]:
final_solutions = generate_solution(X_leaderboard,leaderboard)
final_solutions.pop(0)

100%|██████████| 500/500 [01:36<00:00,  5.16it/s]


[]

In [ ]:
solutions = pd.DataFrame(final_solutions, columns = ['session_id','item_id', 'rank']).reset_index(drop = True)
solutions_csv = solutions.to_csv
solutions.to_csv(r'Simple_LSTM_Solution.csv',index = False)

In [ ]:
solutions[100:150]

,session_id,item_id,rank
100,200,4130,1
101,200,1368,2
102,200,10163,3
103,200,20629,4
104,200,22747,5
105,200,4028,6
106,200,19150,7
107,200,17089,8
108,200,19310,9
109,200,18156,10
